### Install required libraries 

In [1]:

import subprocess
import sys
import os
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

def install_requirements():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while installing packages: {e}")
        sys.exit(1)

if __name__ == "__main__":
    install_requirements()

# Main Program

In [ ]:
# Import required libraries
import cv2
import numpy as np
import mediapipe as mp
from ultralytics import YOLO

# Start with defining the PlayingCardProximityDetector class for card detection
class PlayingCardProximityDetector:
    def __init__(self, model_path):
        self.yolo_model = YOLO(model_path)  # Loading Model
        self.mp_hands = mp.solutions.hands  # MediaPipe Hands
        self.mp_drawing = mp.solutions.drawing_utils
        self.hands = self.mp_hands.Hands(  # MediaPipe Hands
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.4
        )
        
        self.card_details = {  
            # Creating card details dictionary for sharing information in upon user interaction
            
            # Hearts
            '2H': {'name': 'Two of Hearts', 'value': 2, 'suit': 'Hearts', 'color': 'Red'},
            '3H': {'name': 'Three of Hearts', 'value': 3, 'suit': 'Hearts', 'color': 'Red'},
            '4H': {'name': 'Four of Hearts', 'value': 4, 'suit': 'Hearts', 'color': 'Red'},
            '5H': {'name': 'Five of Hearts', 'value': 5, 'suit': 'Hearts', 'color': 'Red'},
            '6H': {'name': 'Six of Hearts', 'value': 6, 'suit': 'Hearts', 'color': 'Red'},
            '7H': {'name': 'Seven of Hearts', 'value': 7, 'suit': 'Hearts', 'color': 'Red'},
            '8H': {'name': 'Eight of Hearts', 'value': 8, 'suit': 'Hearts', 'color': 'Red'},
            '9H': {'name': 'Nine of Hearts', 'value': 9, 'suit': 'Hearts', 'color': 'Red'},
            '10H': {'name': 'Ten of Hearts', 'value': 10, 'suit': 'Hearts', 'color': 'Red'},
            'JH': {'name': 'Jack of Hearts', 'value': 10, 'suit': 'Hearts', 'color': 'Red'},
            'QH': {'name': 'Queen of Hearts', 'value': 10, 'suit': 'Hearts', 'color': 'Red'},
            'KH': {'name': 'King of Hearts', 'value': 10, 'suit': 'Hearts', 'color': 'Red'},
            'AH': {'name': 'Ace of Hearts', 'value': 11, 'suit': 'Hearts', 'color': 'Red'},
            
            # Diamonds
            '2D': {'name': 'Two of Diamonds', 'value': 2, 'suit': 'Diamonds', 'color': 'Red'},
            '3D': {'name': 'Three of Diamonds', 'value': 3, 'suit': 'Diamonds', 'color': 'Red'},
            '4D': {'name': 'Four of Diamonds', 'value': 4, 'suit': 'Diamonds', 'color': 'Red'},
            '5D': {'name': 'Five of Diamonds', 'value': 5, 'suit': 'Diamonds', 'color': 'Red'},
            '6D': {'name': 'Six of Diamonds', 'value': 6, 'suit': 'Diamonds', 'color': 'Red'},
            '7D': {'name': 'Seven of Diamonds', 'value': 7, 'suit': 'Diamonds', 'color': 'Red'},
            '8D': {'name': 'Eight of Diamonds', 'value': 8, 'suit': 'Diamonds', 'color': 'Red'},
            '9D': {'name': 'Nine of Diamonds', 'value': 9, 'suit': 'Diamonds', 'color': 'Red'},
            '10D': {'name': 'Ten of Diamonds', 'value': 10, 'suit': 'Diamonds', 'color': 'Red'},
            'JD': {'name': 'Jack of Diamonds', 'value': 10, 'suit': 'Diamonds', 'color': 'Red'},
            'QD': {'name': 'Queen of Diamonds', 'value': 10, 'suit': 'Diamonds', 'color': 'Red'},
            'KD': {'name': 'King of Diamonds', 'value': 10, 'suit': 'Diamonds', 'color': 'Red'},
            'AD': {'name': 'Ace of Diamonds', 'value': 11, 'suit': 'Diamonds', 'color': 'Red'},
            
            # Clubs
            '2C': {'name': 'Two of Clubs', 'value': 2, 'suit': 'Clubs', 'color': 'Black'},
            '3C': {'name': 'Three of Clubs', 'value': 3, 'suit': 'Clubs', 'color': 'Black'},
            '4C': {'name': 'Four of Clubs', 'value': 4, 'suit': 'Clubs', 'color': 'Black'},
            '5C': {'name': 'Five of Clubs', 'value': 5, 'suit': 'Clubs', 'color': 'Black'},
            '6C': {'name': 'Six of Clubs', 'value': 6, 'suit': 'Clubs', 'color': 'Black'},
            '7C': {'name': 'Seven of Clubs', 'value': 7, 'suit': 'Clubs', 'color': 'Black'},
            '8C': {'name': 'Eight of Clubs', 'value': 8, 'suit': 'Clubs', 'color': 'Black'},
            '9C': {'name': 'Nine of Clubs', 'value': 9, 'suit': 'Clubs', 'color': 'Black'},
            '10C': {'name': 'Ten of Clubs', 'value': 10, 'suit': 'Clubs', 'color': 'Black'},
            'JC': {'name': 'Jack of Clubs', 'value': 10, 'suit': 'Clubs', 'color': 'Black'},
            'QC': {'name': 'Queen of Clubs', 'value': 10, 'suit': 'Clubs', 'color': 'Black'},
            'KC': {'name': 'King of Clubs', 'value': 10, 'suit': 'Clubs', 'color': 'Black'},
            'AC': {'name': 'Ace of Clubs', 'value': 11, 'suit': 'Clubs', 'color': 'Black'},
            
            # Spades
            '2S': {'name': 'Two of Spades', 'value': 2, 'suit': 'Spades', 'color': 'Black'},
            '3S': {'name': 'Three of Spades', 'value': 3, 'suit': 'Spades', 'color': 'Black'},
            '4S': {'name': 'Four of Spades', 'value': 4, 'suit': 'Spades', 'color': 'Black'},
            '5S': {'name': 'Five of Spades', 'value': 5, 'suit': 'Spades', 'color': 'Black'},
            '6S': {'name': 'Six of Spades', 'value': 6, 'suit': 'Spades', 'color': 'Black'},
            '7S': {'name': 'Seven of Spades', 'value': 7, 'suit': 'Spades', 'color': 'Black'},
            '8S': {'name': 'Eight of Spades', 'value': 8, 'suit': 'Spades', 'color': 'Black'},
            '9S': {'name': 'Nine of Spades', 'value': 9, 'suit': 'Spades', 'color': 'Black'},
            '10S': {'name': 'Ten of Spades', 'value': 10, 'suit': 'Spades', 'color': 'Black'},
            'JS': {'name': 'Jack of Spades', 'value': 10, 'suit': 'Spades', 'color': 'Black'},
            'QS': {'name': 'Queen of Spades', 'value': 10, 'suit': 'Spades', 'color': 'Black'},
            'KS': {'name': 'King of Spades', 'value': 10, 'suit': 'Spades', 'color': 'Black'},
            'AS': {'name': 'Ace of Spades', 'value': 11, 'suit': 'Spades', 'color': 'Black'}
        }
        
        self.PROXIMITY_THRESHOLD = 0.15  # Proximity and detection parameters
        self.DETECT_COOLDOWN = 20  # Proximity and detection parameters to avoid multiple detections
        
        self.last_detected_card = None  # Tracking variables for multiple detections
        self.detection_cooldown = 0  # Tracking variables for multiple detections
        self.current_card_details = None  # Tracking variables for multiple detections
        
        self.colors = [  # Defining 52 unique colors for each card
            (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255),
            (128, 0, 0), (0, 128, 0), (0, 0, 128), (128, 128, 0), (128, 0, 128), (0, 128, 128),
            (64, 0, 0), (0, 64, 0), (0, 0, 64), (64, 64, 0), (64, 0, 64), (0, 64, 64),
            (192, 0, 0), (0, 192, 0), (0, 0, 192), (192, 192, 0), (192, 0, 192), (0, 192, 192),
            (255, 128, 0), (255, 0, 128), (128, 255, 0), (0, 255, 128), (128, 0, 255), (0, 128, 255),
            (255, 64, 0), (255, 0, 64), (64, 255, 0), (0, 255, 64), (64, 0, 255), (0, 64, 255),
            (255, 192, 0), (255, 0, 192), (192, 255, 0), (0, 255, 192), (192, 0, 255), (0, 192, 255),
            (128, 128, 128), (192, 192, 192), (64, 64, 64), (255, 255, 255), (0, 0, 0), (128, 128, 255),
            (255, 128, 128), (128, 255, 128)
        ]
    
    def calculate_distance(self, point1, point2, frame_shape):
        # Scaling coordinates to frame size
        x1 = point1.x * frame_shape[1]  
        y1 = point1.y * frame_shape[0]
        x2 = point2.x * frame_shape[1]
        y2 = point2.y * frame_shape[0]
        
        distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)  # Calculate Euclidean distance between object and hand
        
        max_distance = np.sqrt(frame_shape[0]**2 + frame_shape[1]**2)  # Normalize distance to frame size
        normalized_distance = distance / max_distance
        # Returning normalized distance
        return normalized_distance
    # defining a function to calculate the center of the bounding box
    def calculate_box_center(self, bbox, frame_shape):
        x1, y1, x2, y2 = bbox  # Calculate the center point of a bounding box
        center_x = (x1 + x2) / (2 * frame_shape[1])
        center_y = (y1 + y2) / (2 * frame_shape[0])
        # Returning the center of the bounding box
        return type('Point', (), {'x': center_x, 'y': center_y})
    # Defining a function to run the detection
    def run_detection(self):
        cap = cv2.VideoCapture(0)  # Use 0 as argument to Open webcam or give path to video file
        # Using while loop to read the frames
        while cap.isOpened():
            success, frame = cap.read()  # Read frame
            if not success:
                break
            
            frame = cv2.flip(frame, 1)  # Flip frame for mirror effect
            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB for MediaPipe
            
            hand_results = self.hands.process(rgb_frame)  # Detect hands
            
            yolo_results = self.yolo_model(frame)[0]  # Run YOLO detection
            
            boxes = yolo_results.boxes.cpu()  # Get bounding boxes and class names
            
            if self.detection_cooldown > 0:  # Decrement detection cooldown
                self.detection_cooldown -= 1
            
            for box in boxes:  # Drawing YOLO detections
                x1, y1, x2, y2 = box.xyxy[0]  # Bounding box
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                conf = box.conf[0]  # Confidence and class
                cls = int(box.cls[0])
                class_name = self.yolo_model.names[cls]
                
                color = self.colors[cls % len(self.colors)]  # Define a color based on the class index
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Drawing bounding box with unique color
                label = f'{class_name} {conf:.2f}'
                cv2.putText(frame, label, (x1, y1-10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2) #  Seting the font and size of the text for lable
            
            if hand_results.multi_hand_landmarks:  # Process hand landmarks
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    self.mp_drawing.draw_landmarks(  # Drawing hand landmarks
                        frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
                    
                    index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]  # Get index finger tip
                    
                    close_cards = []  # Track closest cards
                    
                    for box in boxes:  # Check proximity to cards
                        x1, y1, x2, y2 = box.xyxy[0]
                        cls = int(box.cls[0])
                        class_name = self.yolo_model.names[cls]
                        
                        box_center = self.calculate_box_center(  # Calculate box center
                            (x1, y1, x2, y2), 
                            frame.shape
                        )
                        
                        distance = self.calculate_distance(index_tip, box_center, frame.shape)  # Calculate distance
                        
                        if distance < self.PROXIMITY_THRESHOLD:  # Checking if within proximity threshold
                            close_cards.append((class_name, distance))
                    
                    close_cards.sort(key=lambda x: x[1])  # Sort close cards by distance (closest first)
                    
                    if close_cards and self.detection_cooldown == 0:  # Handle detection with cooldown and multiple close cards
                        closest_card = close_cards[0][0]  # Get the closest card
                        
                        if closest_card != self.last_detected_card:  # Check if it's a different card
                            card_info = self.card_details.get(closest_card, {'name': 'Unknown Card'})  # Retrieve card details
                            
                            self.current_card_details = [  # Prepare detailed information
                                f"Name: {card_info.get('name', 'N/A')}",
                                f"Value: {card_info.get('value', 'N/A')}",
                                f"Suit: {card_info.get('suit', 'N/A')}",
                                f"Color: {card_info.get('color', 'N/A')}"
                            ]
                            
                            self.last_detected_card = closest_card  # Update tracking variables
                            self.detection_cooldown = self.DETECT_COOLDOWN
                    
                    if self.current_card_details:  # Display persistent card details if available
                        cv2.putText(frame, "CARD DETECTED",  # Visual feedback for proximity
                                    (10, frame.shape[0] - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                                    (0, 255, 0), 2)
                        
                        for i, detail in enumerate(self.current_card_details):  # Display card details
                            cv2.putText(frame, detail, 
                                        (10, frame.shape[0] - 50 - (i * 30)), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                                        (255, 255, 255), 2)
            
            cv2.imshow('Playing Card Proximity Detection', frame)  # Display the frame
            
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Break loop on 'q' press
                break
        
        cap.release()
        cv2.destroyAllWindows()

# Define path to model file
detector = PlayingCardProximityDetector('best.pt')
# Run Inference
detector.run_detection()


/Users/jayna/Downloads/program/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
I0000 00:00:1734432339.591525 1141385 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1734432339.596781 1141653 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734432339.600939 1141665 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-17 11:45:39.661 Python[13766:1141385] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCamera


0: 384x640 (no detections), 19.2ms
Speed: 1.7ms preprocess, 19.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 17.4ms
Speed: 1.2ms preprocess, 17.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.8ms
Speed: 1.1ms preprocess, 15.8ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.7ms
Speed: 1.1ms preprocess, 19.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 17.1ms
Speed: 1.3ms preprocess, 17.1ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.0ms
Speed: 1.2ms preprocess, 16.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.2ms
Speed: 1.2ms preprocess, 15.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.4ms
Speed: 1.0ms preprocess, 15.4ms i

W0000 00:00:1734432341.752272 1141652 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2024-12-17 11:45:41.774 Python[13766:1141385] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-17 11:45:41.774 Python[13766:1141385] +[IMKInputSession subclass]: chose IMKInputSession_Modern



0: 384x640 (no detections), 15.5ms
Speed: 1.2ms preprocess, 15.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.4ms
Speed: 1.2ms preprocess, 14.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.8ms
Speed: 1.1ms preprocess, 16.8ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.8ms
Speed: 1.2ms preprocess, 16.8ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.7ms
Speed: 1.0ms preprocess, 14.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.5ms
Speed: 1.2ms preprocess, 14.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.2ms
Speed: 1.3ms preprocess, 15.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.5ms
Speed: 1.4ms preprocess, 14.5ms i